In [8]:
library(Seurat)
library(patchwork)
library(ggpubr)
library(harmony)
library(dplyr)

Loading required package: Rcpp



In [ ]:
setwd('./')
CL <- c("18Y.CL2","18Y.CL-E","18Y.control","27Y.CL-E","27Y.control","32Y.CL-L","32Y.control","47Y.CL-L-1","47Y.CL-L-2","47Y.control","49Y.CL-L","49Y.control",'32y.CA','40y.CA','42y.CA')
CL2 <- c("18Y-E1","18Y-E2","18Y-C","27Y-E","27Y-C","32Y-L","32Y-C","47Y-L1","47Y-L2","47Y-C","49Y-L","49Y-C",'32y.CA','40y.CA','42y.CA')

scRNAlist = {}
for(i in (1:length(CL))){
     file=paste0("./",CL[i])
     data <- Read10X(data.dir = CL[i])
     scRNAlist[[i]] <- CreateSeuratObject(counts = data,project = CL2[i] , min.cells = 0, min.features = 0)
     print(CL[i])
}

scRNA=scRNAlist[[1]]
for (j in (2:length(scRNAlist))){
    scRNA <- merge(scRNA,scRNAlist[j])
}
saveRDS(scRNA, file ="CL&CA.raw.rds")

In [4]:
table(scRNA@meta.data$orig.ident)


 18Y-C 18Y-E1 18Y-E2  27Y-C  27Y-E  32Y-C  32Y-L 32y.CA 40y.CA 42y.CA  47Y-C 
   600     95    250    518    214   1311    305    416    511    369   1003 
47Y-L1 47Y-L2  49Y-C  49Y-L 
   186    480   1483    933 

In [6]:
scRNA@meta.data$Stage <- scRNA@meta.data$orig.ident
scRNA@meta.data=scRNA@meta.data %>% mutate(Stage=case_when(
                                            Stage == "18Y-E1" ~ "Corpus luteum",
                                            Stage == "18Y-E2" ~ "Corpus luteum", 
                                            Stage == "27Y-E" ~ "Corpus luteum",
                                            Stage == "32Y-L" ~ "Luteolysis",
                                            Stage == "32y.CA" ~ "Corpus albicans",
                                            Stage == "40y.CA" ~ "Corpus albicans",
                                            Stage == "42y.CA" ~ "Corpus albicans",
                                            Stage == "47Y-L1" ~ "Luteolysis",
                                            Stage == "47Y-L2" ~ "Luteolysis",
                                            Stage == "49Y-L" ~ "Luteolysis",
                                            Stage == "18Y-C" ~ "Background",
                                            Stage == "27Y-C" ~ "Background",
                                            Stage == "32Y-C" ~ "Background",
                                            Stage == "47Y-C" ~ "Background",
                                            Stage == "49Y-C" ~ "Background",
                                            
                         TRUE ~ NA_character_))

In [7]:
unique(scRNA@meta.data$Stage)

[1] "Corpus luteum"   "Background"      "Luteolysis"      "Corpus albicans"

In [11]:
scRNA@meta.data$Stage <- factor(scRNA@meta.data$Stage,levels=c("Corpus luteum","Luteolysis","Corpus albicans","Background"))

In [18]:
scRNA <- NormalizeData(scRNA, normalization.method = "LogNormalize", scale.factor = 10000)
scRNA <- PercentageFeatureSet(scRNA, '^MT-', col.name = "percent.mt")
scRNA <- PercentageFeatureSet(scRNA, '^RP[SL]', col.name = "percent.ribo")
scRNA <- PercentageFeatureSet(scRNA, '^HB[^(P)]', col.name = "percent.hb")

In [ ]:
scRNA <- FindVariableFeatures(scRNA, selection.method = "vst", nfeatures = 5000)
scRNA <- ScaleData(scRNA,vars.to.regress =c("percent.mt","nCount_RNA","orig.ident"))
scRNA <- RunPCA(scRNA,features = VariableFeatures(object =scRNA),npcs=50, verbose=FALSE)
scRNA <- RunHarmony(scRNA, group.by.vars="orig.ident", assay.use="RNA", max.iter.harmony = 20)
scRNA <- RunUMAP(scRNA, reduction = "harmony", dims = 1:30,min.dist =0.05,spread =10)
scRNA <- RunTSNE(scRNA, reduction = "harmony", dims = 1:30)
scRNA <- FindNeighbors(scRNA, reduction="harmony", dims = 1:30)

In [20]:
saveRDS(scRNA, file ="CL&CA.final.rds")

In [28]:
library(tidyr)
meta <- FetchData(scRNA, vars='orig.ident', slot = "data")
meta$celltype = rownames(meta)

In [ ]:
meta_new <- separate(meta, col = celltype, into = c("celltype","orig"), sep = '_')
meta_new <- separate(meta_new, col = celltype, into = c("No1","celltype", "No2"), sep = '-')
meta_new$new_celltype <- paste0(meta_new$No1,'_',meta_new$celltype)
meta_new = meta_new[,c('orig.ident','new_celltype')]
scRNA <- AddMetaData(scRNA, metadata = meta_new$new_celltype, col.name = 'celltype')

In [ ]:
colour= c('gray','black','blue','purple','#FF7F00','green','red')
mytheme = theme(plot.title = element_text(size = 20,color="black",hjust = 0.5),
                 axis.title = element_text(size =24,color ="black"),
                 axis.text = element_text(size=18,color = "black"),
                 panel.grid.minor.y = element_blank(),
                 panel.grid.minor.x = element_blank(),
                  panel.grid=element_blank(), # 去网格线
                 legend.text = element_text(size=14),
                 legend.title= element_text(size= 18),
                 axis.text.x = element_text(angle = 45, hjust=1, vjust=1))
sample_table <- as.data.frame(table(sce@meta.data$Stage,sce@meta.data$celltype))
names(sample_table) <- c("Stage","Celltype","CellNumber")
sample_table$Stage <- factor(sample_table$Stage,levels=c("Corpus luteum","Luteolysis","Corpus albicans","Background"))
plot_sample1<-ggplot(sample_table,aes(x=Stage,weight=CellNumber,fill=Celltype))+
  geom_bar(position="fill")+ylab("Percentage")+theme_bw() +
  scale_fill_manual(values=colour) +mytheme
plot_sample2<-ggplot(sample_table,aes(x=Stage,weight=CellNumber,fill=Celltype))+
  geom_bar()+ylab("Number")+theme_bw() +
  scale_fill_manual(values=colour) +mytheme

options(repr.plot.width =10,repr.plot.height =6)
p=CombinePlots(
  plots = list(plot_sample1,plot_sample2),
  nrow =1
)
p
ggsave("CL&CA.stage.celltype.barplot.pdf",plot=p,w=10,h=6)

In [ ]:
# cellcycle analysis
scRNA<- CellCycleScoring(
  object = scRNA,
  g2m.features = cc.genes$g2m.genes,
  s.features = cc.genes$s.genes
)
colour=c('yellow4', 'steelblue4', 'darkturquoise',
                    'green1', 'yellow4', 'yellow3','darkorange4', 'brown')
sample_table <- as.data.frame(table(sce@meta.data$Stage,sce@meta.data$Phase))
names(sample_table) <- c("Samples","Phase","CellNumber")
head(sample_table)
sample_table$Samples <- factor(sample_table$Samples,levels=c("Corpus luteum","Luteolysis","Corpus albicans","Background"))
plot_sample1<-ggplot(sample_table,aes(x=Samples,weight=CellNumber,fill=Phase))+
  geom_bar(position="fill")+ylab("Percentage")+theme_bw() +
  scale_fill_manual(values=colour) +mytheme
plot_sample2<-ggplot(sample_table,aes(x=Samples,weight=CellNumber,fill=Phase))+
  geom_bar()+ylab("Number")+theme_bw() +
  scale_fill_manual(values=colour) +mytheme

options(repr.plot.width =12,repr.plot.height =6)
p=CombinePlots(
  plots = list(plot_sample1,plot_sample2),
  nrow =1
)
p
ggsave("CL&CA.stage.cellcycle.barplot.pdf",plot=p,w=10,h=6)